By default `ptplot` will produce a single plot with all your data on it. However, to show multiple plots in a single visualization you can add grouping logic in a `Facet` layer.

_(NOTE: If you are viewing this notebook on GitHub, none of the plots will render due to the [way GitHub processes the notebooks](https://github.com/plotly/plotly.py/issues/931). To see the notebook rendered properly, please [use nbviewer](https://nbviewer.jupyter.org/github/AndrewRook/ptplot/blob/main/notebooks/2-Animations.ipynb).)_

Starting with imports, as usual. 

In [1]:
import numpy as np
import pandas as pd

from ptplot import PTPlot
from ptplot.animation import Animation
from ptplot.hover import Hover
from ptplot.facet import Facet
from ptplot.nfl import Aesthetics, Field
from ptplot.plot import Positions, Tracks

from bokeh.plotting import show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
# Load the data file
player_tracking_data = pd.read_csv(
    "2018_CLE_2018122305_1246.tsv",
    sep="\t", parse_dates=["time"]
)
player_tracking_data.loc[player_tracking_data["displayName"] == "ball", "teamAbbr"] = "ball"
player_tracking_data.loc[player_tracking_data["displayName"] == "ball", "jerseyNumber"] = ""

# Just look between the snap and the tackle, to make the tracks clearer
snap_frame = player_tracking_data[player_tracking_data["event"] == "ball_snap"]["frame"].unique()[0]
tackle_frame = player_tracking_data[player_tracking_data["event"] == "tackle"]["frame"].unique()[0]
player_tracking_data = player_tracking_data[
    player_tracking_data["frame"].between(snap_frame, tackle_frame)
]
player_tracking_data.frame = player_tracking_data.frame - player_tracking_data.frame.min()

This data comes with several frames marked as belonging to specific game events. We can use facets to see positions at every event between the snap and the tackle simultaneously:

In [3]:
plot = (
    PTPlot(data=player_tracking_data, pixel_height=800) 
    + Field() 
    + Positions(
        "x", "y", number="jerseyNumber",
        name="positions" # The name is used to map the Hover label to a layer
    )
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
    + Hover([("name", "@displayName")], "positions", ["displayName"])
    + Facet("event", num_col=2)
)
    
show(plot.draw())

We can use facets with animations to do things like show plays side-by-side. We'll grab another play from the same repo as the first one (specifically [here](https://github.com/asonty/ngs_highlights/blob/20f74138f0a2f7e91879d31bbf2d8addc0bd59bc/play_data/2018_CLE_2018120902_365.tsv)), synchronize the frames to start at the snap, and run them at the same time.

In [12]:
# Grabbing another play from the same repo:
second_play_data = pd.read_csv(
    "2018_GB_2018090912_3564.tsv",
    sep="\t", parse_dates=["time"]
)
second_play_data.loc[second_play_data["displayName"] == "ball", "teamAbbr"] = "ball"
second_play_data.loc[second_play_data["displayName"] == "ball", "jerseyNumber"] = ""
snap_frame = second_play_data[second_play_data["event"] == "ball_snap"]["frame"].unique()[0]
tackle_frame = second_play_data[second_play_data["event"] == "touchdown"]["frame"].unique()[0]
second_play_data = second_play_data[
    second_play_data["frame"].between(snap_frame, tackle_frame)
]
second_play_data.frame = second_play_data.frame - second_play_data.frame.min()

In [14]:
combined_play_data = pd.concat([player_tracking_data, second_play_data])

plot = (
    PTPlot(data=combined_play_data, pixel_height=300) 
    + Field() 
    + Positions(
        "x", "y", number="jerseyNumber",
        name="positions" # The name is used to map the Hover label to a layer
    )
    + Aesthetics(team_ball_mapping="teamAbbr", home_away_mapping="homeTeamFlag == 1", ball_identifier="ball")
    + Hover([("name", "@displayName")], "positions", ["displayName"])
    + Facet("playDescription", num_col=1)
    + Animation("frame")
)
    
show(plot.draw())